In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import matplotlib.pyplot as plt 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

/tmp/ipykernel_746/2878713752.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-02-29 23:16:42.191037: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 23:16:42.191284: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 23:16:42.558003: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to

In [2]:
def load_dataset(csv_path):
    df = pd.read_csv(csv_path, sep= ';')
    required_columns = ['Azimuth', 'Altitude', 'Timestamp']

    if not all (col in df.columns for col in required_columns):
        raise ValueError("Required columns are not present in the DataFrame.")
    
    X = df[['Timestamp','Azimuth','Altitude']].values
    y = df['Azimuth'].values

    return X, y, df

In [3]:
csv_path = 'cleaned/cleaned_data_quartal_1_2020.csv'
X, y, df = load_dataset(csv_path)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 , random_state= 42)

In [5]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))

In [6]:
timesteps = 1

In [7]:
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], timesteps, X_test_scaled.shape[1]))

In [10]:
model = Sequential()
model.add(LSTM(units=2048, activation='relu', input_shape=(timesteps, X_train_scaled.shape[1]), return_sequences=True))
model.add(LSTM(units=512, activation= 'relu', return_sequences=True))
model.add(LSTM(units=256, activation= 'relu', return_sequences=True))
model.add(LSTM(units=128, activation= 'relu', return_sequences=True))
model.add(LSTM(units=64, activation= 'relu', return_sequences=True))
model.add(LSTM(units=32, activation= 'relu'))
model.add(Dense(units=1))

optimizer = Adam(learning_rate= 0.0001)
model.compile(optimizer=optimizer, loss = 'mse')

model.fit(X_train_reshaped, y_train_scaled, epochs= 500, batch_size = 256)

Epoch 1/500
41/41 [==============================] - 7s 47ms/step - loss: 0.2879
Epoch 2/500
41/41 [==============================] - 2s 46ms/step - loss: 0.2800
Epoch 3/500
41/41 [==============================] - 2s 48ms/step - loss: 0.2711
Epoch 4/500
41/41 [==============================] - 2s 47ms/step - loss: 0.2561
Epoch 5/500
41/41 [==============================] - 2s 46ms/step - loss: 0.1414
Epoch 6/500
41/41 [==============================] - 2s 46ms/step - loss: 0.0348
Epoch 7/500
41/41 [==============================] - 2s 44ms/step - loss: 0.0229
Epoch 8/500
41/41 [==============================] - 2s 45ms/step - loss: 0.0127
Epoch 9/500
41/41 [==============================] - 2s 46ms/step - loss: 0.0045
Epoch 10/500
41/41 [==============================] - 2s 46ms/step - loss: 0.0011
Epoch 11/500
41/41 [==============================] - 2s 46ms/step - loss: 5.7541e-04
Epoch 12/500
41/41 [==============================] - 2s 46ms/step - loss: 5.0492e-04
Epoch 13/500
41/4

In [9]:
predicted_values_scaled = model.predict(X_test_reshaped)
predicted_values = scaler_y.inverse_transform(predicted_values_scaled)
mse = mean_squared_error(y_test_scaled, predicted_values_scaled)
r2 = r2_score(y_test_scaled, predicted_values_scaled)
accuracy = model.evaluate(X_test_reshaped, y_test_scaled)
print(f"Mean Squared Error on Test Data: {mse}")
print(f"R-squared on Test Data: {r2}")
print(f"Accuracy on Test Data: {accuracy}")


82/82 [==============================] - 1s 5ms/step - loss: 1.8298e-07
Mean Squared Error on Test Data: 1.8297913521783408e-07
R-squared on Test Data: 0.9999956702791117
Accuracy on Test Data: 1.829795195362749e-07
